# Introdução à Banco de Dados | Trabalho Prático III
Abel Severo Rocha, Ana Carla Fernandes, Natasha Caxias

Prof. Dr. Altigran Soares da Silva

## Parte I
Verificação dos parâmetros do hardware e do software utilizado e familiarização com o ambiente do PostgreSQL.

### Tarefa 1 - Identificação do Sistema

In [ ]:
! pip install pandas tabulate

In [29]:
import platform
import time
import random
import threading
import matplotlib.pyplot as plt
import threading
import os
import shutil
import subprocess
import datetime
import psycopg2
import pandas as pd
from tabulate import tabulate
from psycopg2.extras import RealDictCursor
from tpch_pgsql import main as tpch_pgsql
from tpch4pgsql import postgresqldb as pgdb, result as r

In [2]:
def mostrarSoInfo():
    
    soInfo = platform.freedesktop_os_release()
    print("\n--------- Sistema Operacional (SO) ---------\n")
    print("Sistema Operacional ", platform.system())
    print("Versão              ", soInfo["VERSION"])
    print("Release (kernel)    ", platform.release())
    print("Arquitetura         ", platform.machine())
    print("Distribuição        ", soInfo["NAME"])


In [3]:
def mostrarHardwareInfo():

    cpuInfo = {}
    with open("/proc/cpuinfo") as f:
        for line in f:
            if ":" in line:
                k, v = line.split(":", 1)
                cpuInfo[k.strip()] = v.strip()
    
    ramInfo = {}
    with open("/proc/meminfo") as f:
        for line in f:
            k, v = line.split(":", 1)
            ramInfo[k] = v.strip()

    print("\n------------- Sobre o Hardware -------------\n")
    print("Processador:")
    print("  • Modelo          ", cpuInfo["model name"])
    print("  • CPUs lógicas    ", os.cpu_count())
    print("  • Clock base      ", cpuInfo["cpu MHz"], "MHz")

    print("\nCache:")
    with open("/sys/devices/system/cpu/cpu0/cache/index0/size") as f:
        print("  • L1d             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index1/size") as f:
        print("  • L1i             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index2/size") as f:
        print("  • L2              ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index3/size") as f:
        print("  • L3              ", f.read().strip()[:-1]+" kB")

    print("\nMemória RAM:")
    print("  • Total           ", ramInfo["MemTotal"])
    print("  • Livre           ", ramInfo["MemFree"])
    print("  • Disponível      ", ramInfo["MemAvailable"])

    total, usado, livre = shutil.disk_usage("/")
    print("\nDisco:")
    print(f"  • Total            {total/1024**3:.2f} GB")
    print(f"  • Usado            {usado/1024**3:.2f} GB")
    print(f"  • Livre            {livre/1024**3:.2f} GB")


In [4]:
mostrarSoInfo()
mostrarHardwareInfo()


--------- Sistema Operacional (SO) ---------

Sistema Operacional  Linux
Versão               24.04.3 LTS (Noble Numbat)
Release (kernel)     6.8.0-87-generic
Arquitetura          x86_64
Distribuição         Ubuntu

------------- Sobre o Hardware -------------

Processador:
  • Modelo           11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz
  • CPUs lógicas     8
  • Clock base       1300.000 MHz

Cache:
  • L1d              48 kB
  • L1i              32 kB
  • L2               1280 kB
  • L3               12288 kB

Memória RAM:
  • Total            7670432 kB
  • Livre            403028 kB
  • Disponível       1787132 kB

Disco:
  • Total            233.18 GB
  • Usado            54.51 GB
  • Livre            166.75 GB


### Tarefa 2 - Verificação de parâmetros de armazenamento

In [5]:
BASE = "/sys/block"

def run(cmd):
    return subprocess.check_output(cmd, text=True).strip()

def getDispositivo():
    for dev in os.listdir(BASE):
        if dev.startswith(("loop", "ram", "dm-")): # ignora
            continue
        return dev # encontra dispositivo
    return None

def ehHD(dispositivo):
    rotational_path = os.path.join(BASE, dispositivo, "queue/rotational")
    
    if os.path.exists(rotational_path):
        with open(rotational_path) as f:
            return f.read().strip() == "1"
    return False

def blocos(dispositivo):
    cmd = f"cat /sys/block/{dispositivo}/queue/logical_block_size"
    print("Blocos Lógicos        ", run(cmd.split()))

    cmd = f"cat /sys/block/{dispositivo}/queue/physical_block_size"
    print("Blcocos Físicos       ", run(cmd.split()))

def stat(dispositivo):
    cmd = f"stat -f /dev/{dispositivo}"
    print("\nParâmetros do SO para o disco (stat):")
    print(run(cmd.split()))

def mostrarDiscoInfo():
    disp = getDispositivo()
    print("Tipos de dispositivo  ", "HD" if ehHD(disp) else "SSD")
    cmd = ["cat", f"/sys/block/{disp}/device/model"]
    print("Modelo                ", run(cmd))
    stat(disp)
    print()
    blocos(disp)
    print()

In [6]:
mostrarDiscoInfo()

Tipos de dispositivo   SSD
Modelo                 SSSTC CL1-4D256

Parâmetros do SO para o disco (stat):
File: "/dev/nvme0n1"
    ID: 303f239e77c82038 Namelen: 255     Type: tmpfs
Block size: 4096       Fundamental block size: 4096
Blocks: Total: 948744     Free: 948744     Available: 948744
Inodes: Total: 948744     Free: 948032

Blocos Lógicos         512
Blcocos Físicos        512



#### Como alterar tamanho dos blocos

Só é possível alterar o tamanho dos blocos **recriando o sistema de arquivos** com o comando 

``sudo mkfs.ext4 -b [tamanho do bloco] /dev/sdXN``

#### Espeficiações do modelo


* Número de superfícies
* Cilindros
* Setores por trilha
* Velocidade de rotação
* Latência rotacional
* Tempo de seek
    * Mínimo
    * Médio
    * Máximo
* Tempo para próxima trilha


### Tarefa 3 – Geração de um BD para testes


Necessário dar permissão de escrita para a pasta _tpch4pgsql_:

``chmod -R u+rwX,g+rwX,o+rX .``

#### Preparar
A fase de preparação compila o TPC-H dbgen e querygen e cria os arquivos de carga e atualização (atualização/exclusão).

In [8]:
tpch_pgsql(phase="prepare")

make: Nothing to be done for 'all'.
built dbgen from source


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating data for suppliers table
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

generated data for the load phase


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating update pair #1 for orders/lineitem tables
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

generated data for the update phase
generated data for the delete phase
created data files in ./data
created query files in ./query_root


done.
Generating update pair #3 for orders/lineitem tablesdone.


#### Carregamento

A fase de carregamento (load) limpa o banco de dados (se necessário), carrega as tabelas no banco de dados e cria índices para consultas. Os resultados desta fase consistem nas seguintes métricas:

* Tempo de criação do esquema
* Tempo de carregamento dos dados
* Tempo de criação de restrições de chave estrangeira e índices

In [9]:
tpch_pgsql(phase="load")

dropped existing tables
cleaned database tpch
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.045028
load_data: 0:00:27.355545
index_tables: 0:00:27.284134
======================= End Results ========================


#### Consultas

Nessa fase, as consultas serão analizadas com o comando ``EXPLAIN ANALYZE``, evidenciando:

* Tempo de execução do planejamento;
* Tempo de execução do _exlpain_;
* Algoritmos utilizados.

Em seguida, as consultas serão executas, exibindo as seguintes informações:

* Até 10 linhas do resultado da consulta;
* Total de linhas encontradas;
* Tempo de execução.

In [10]:
class PGDBW(pgdb.PGDB):

    def __init__(self, host, port, database, user, password):
        # Chamar o construtor da classe pai corretamente
        self.conn = psycopg2.connect(
                    host=host,
                    port=port,
                    dbname=database,
                    user=user,
                    password=password
                ) 
        if hasattr(self, 'conn') and self.conn:
            print("Conexão estabelecida com sucesso!")
        else:
            print("Atenção: Conexão não estabelecida")
    
    def executeQueryFromFileWithResults(self, filepath):
        """Executa query de arquivo e retorna resultados"""
        try:
            # Verificar se a conexão existe
            if not hasattr(self, 'conn') or not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with open(filepath, 'r') as f:
                query = f.read()
            
            # Usar cursor que retorna dicionários
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}
        
    def executeQuery(self, query_string):
        """Executa uma query diretamente a partir de string"""
        try:
            if not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query_string)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    self.conn.commit()
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}

In [11]:
host = "localhost"
port = 5432
user = "postgres"
password = "test123"
database = "tpch"
filepath = "query_root/perf_query_gen/"

conn = PGDBW(host, port, database, user, password)

# Definir algoritmos de junção e varredura comuns no PostgreSQL
algoritmos_juncao = ['Nested Loop', 'Hash Join', 'Merge Join']
algoritmos_varredura = ['Seq Scan', 'Index Scan', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan']
algoritmos_ordenacao = ['Sort', 'Incremental Sort']
algoritmos_agregacao = ['HashAggregate', 'GroupAggregate']

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"EXPLAIN ANALYZE - QUERY {i}")
    print(f"{'=' * 120}")
    
    # Ler a query original
    with open(filepath + f"{i}.sql", 'r') as f:
        original_query = f.read()
    
    # Adicionar EXPLAIN ANALYZE
    explain_query = "EXPLAIN ANALYZE " + original_query
    
    start_time = datetime.datetime.now()
    result = conn.executeQuery(explain_query)
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro no EXPLAIN ANALYZE {i}: {result['error']}")
    elif 'data' in result:
        print(f"\nTempo de execução do EXPLAIN: {execution_time}")
        
        # Coletar algoritmos utilizados
        algoritmos_encontrados = {
            'juncao': [],
            'varredura': [],
            'ordenacao': [],
            'agregacao': [],
            'outros': []
        }
        
        print(f"\nPLANO DE EXECUÇÃO (EXPLAIN ANALYZE):")
        for idx, row in enumerate(result['data']):
            plan_line = list(row.values())[0] if row else ""
            print(f"{plan_line}")
            
            # Identificar algoritmos na linha do plano
            for algoritmo in algoritmos_juncao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['juncao']:
                        algoritmos_encontrados['juncao'].append(algoritmo)
            
            for algoritmo in algoritmos_varredura:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['varredura']:
                        algoritmos_encontrados['varredura'].append(algoritmo)
            
            for algoritmo in algoritmos_ordenacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['ordenacao']:
                        algoritmos_encontrados['ordenacao'].append(algoritmo)
            
            for algoritmo in algoritmos_agregacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['agregacao']:
                        algoritmos_encontrados['agregacao'].append(algoritmo)
        
        # Mostrar resumo dos algoritmos utilizados
        print(f"\n{'─' * 80}")
        print("ALGORITMOS IDENTIFICADOS NO PLANO DE EXECUÇÃO:")
        print(f"{'─' * 80}")
        
        if algoritmos_encontrados['juncao']:
            print(f"Algoritmos de Junção: {', '.join(algoritmos_encontrados['juncao'])}")
        
        if algoritmos_encontrados['varredura']:
            print(f"Algoritmos de Varredura: {', '.join(algoritmos_encontrados['varredura'])}")
        
        if algoritmos_encontrados['ordenacao']:
            print(f"Algoritmos de Ordenação: {', '.join(algoritmos_encontrados['ordenacao'])}")
        
        if algoritmos_encontrados['agregacao']:
            print(f"Algoritmos de Agregação: {', '.join(algoritmos_encontrados['agregacao'])}")
        
        # Verificar se não foram encontrados algoritmos conhecidos
        total_algoritmos = (len(algoritmos_encontrados['juncao']) + 
                          len(algoritmos_encontrados['varredura']) + 
                          len(algoritmos_encontrados['ordenacao']) + 
                          len(algoritmos_encontrados['agregacao']))
        
        if total_algoritmos == 0:
            print("ℹNenhum algoritmo específico identificado (possivelmente plano simples)")
            
    else:
        print(f"{result.get('message', 'Resultado inesperado')}")
        print(f"Tempo: {execution_time}")

Conexão estabelecida com sucesso!

EXPLAIN ANALYZE - QUERY 1

Tempo de execução do EXPLAIN: 0:00:05.681207

PLANO DE EXECUÇÃO (EXPLAIN ANALYZE):
Limit  (cost=346484.38..346485.51 rows=1 width=248) (actual time=5640.834..5640.929 rows=1 loops=1)
  ->  Finalize GroupAggregate  (cost=346484.38..391574.69 rows=40000 width=248) (actual time=5626.697..5626.791 rows=1 loops=1)
        Group Key: l_returnflag, l_linestatus
        ->  Gather Merge  (cost=346484.38..387874.69 rows=80000 width=248) (actual time=5597.751..5626.699 rows=4 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Partial GroupAggregate  (cost=345484.36..377640.68 rows=40000 width=248) (actual time=3962.098..5015.652 rows=3 loops=3)
                    Group Key: l_returnflag, l_linestatus
                    ->  Sort  (cost=345484.36..347568.11 rows=833502 width=144) (actual time=3392.735..3665.980 rows=1440376 loops=3)
                          Sort Key: l_returnflag, l_linestat

In [16]:

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"QUERY {i}")
    print(f"{'=' * 120}")
    
    # Ler a query original
    with open(filepath + f"{i}.sql", 'r') as f:
        original_query = f.read()
    
    start_time = datetime.datetime.now()
    result = conn.executeQuery(original_query)
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro ao executar query {i}: {result['error']}")
        continue
    
    if 'data' not in result or len(result['data']) == 0:
        print("Nenhum dado retornado.")
        continue
    
    rows = result['data']
    
    # Pegar colunas (as chaves do primeiro registro)
    headers = list(rows[0].keys())
    
    # Converter registros para lista de listas
    table = [list(r.values()) for r in rows[:10]]
    
    print(tabulate(table, headers=headers, tablefmt="grid"))
    print(f"Tempo de execução: {execution_time}")



QUERY 1
+----------------+----------------+-------------+------------------+------------------+--------------+-----------+-------------+------------+---------------+
| l_returnflag   | l_linestatus   |     sum_qty |   sum_base_price |   sum_disc_price |   sum_charge |   avg_qty |   avg_price |   avg_disc |   count_order |
+================+================+=============+==================+==================+==============+===========+=============+============+===============+
| A              | F              | 3.77341e+07 |      5.65866e+10 |      5.37583e+10 |  5.59091e+10 |    25.522 |     38273.1 |  0.0499853 |       1478493 |
+----------------+----------------+-------------+------------------+------------------+--------------+-----------+-------------+------------+---------------+
Tempo de execução: 0:00:02.567576

QUERY 3
+--------------+-----------+---------------+------------------+
|   l_orderkey |   revenue | o_orderdate   |   o_shippriority |
+==============+===========+==

## Parte II
Análise do comportamento dos índices das tabelas do SGBD através do exame das tabelas de estatísticas para consultas SQL.

In [17]:
# Configuração de conexão para a PARTE II
DB_CONFIG_PARTE2 = {
    "dbname": "tpch",
    "user": "postgres",
    "password": "test123",   
    "host": "localhost",
    "port": 5432
}

conn_p2 = psycopg2.connect(**DB_CONFIG_PARTE2)
cur_p2 = conn_p2.cursor()
print("Conectado ao PostgreSQL.")


Conectado ao PostgreSQL.


### Tarefa 5 – Preparação da Tabela Aleatória



In [18]:
cur_p2.execute("""
DROP TABLE IF EXISTS t;
CREATE TABLE t (
    k serial PRIMARY KEY,
    v integer
);

INSERT INTO t(v)
SELECT trunc(random() * 10)
FROM generate_series(1,100000);
""")
conn_p2.commit()
print("Tabela t criada e populada com 100.000 tuplas.")

cur_p2.execute("SELECT * FROM t ORDER BY k LIMIT 10;")
rows = cur_p2.fetchall()
df_t10 = pd.DataFrame(rows, columns=["k", "v"])
print("\nPrimeiras 10 tuplas da tabela t (ordenadas por k):")
print(df_t10)

Tabela t criada e populada com 100.000 tuplas.

Primeiras 10 tuplas da tabela t (ordenadas por k):
    k  v
0   1  1
1   2  5
2   3  0
3   4  7
4   5  8
5   6  6
6   7  3
7   8  7
8   9  7
9  10  9


### Tarefa 6 – Páginas criadas

In [20]:
cur_p2.execute("""
SELECT relname, relpages, reltuples
FROM pg_class
WHERE relname = 't';
""")
rows = cur_p2.fetchall()
df_pgclass = pd.DataFrame(rows, columns=["relname", "relpages", "reltuples"])
print(df_pgclass)

  relname  relpages  reltuples
0       t         0       -1.0


### Tarefa 7 – Blocos

In [21]:
cur_p2.execute("SELECT pg_sleep(1);")

cur_p2.execute("""
SELECT *
FROM pg_stats
WHERE tablename = 't';
""")
cols = [desc[0] for desc in cur_p2.description]
df_pgstats = pd.DataFrame(cur_p2.fetchall(), columns=cols)

print("\nEstatísticas em pg_stats para tabela t:")
print(df_pgstats)

cur_p2.execute("SELECT pg_stat_reset();")
conn_p2.commit()
print("\nEstatísticas resetadas com pg_stat_reset().")


Estatísticas em pg_stats para tabela t:
Empty DataFrame
Columns: [schemaname, tablename, attname, inherited, null_frac, avg_width, n_distinct, most_common_vals, most_common_freqs, histogram_bounds, correlation, most_common_elems, most_common_elem_freqs, elem_count_histogram]
Index: []

Estatísticas resetadas com pg_stat_reset().


### Tarefa 8 – Índice

In [22]:
# Índice e consulta com 100k tuplas
inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v ON t(v);")
conn_p2.commit()
fim = time.time()
tempo_criacao_100k = fim - inicio
print(f"Tempo para criar índice idx_v (100k tuplas): {tempo_criacao_100k:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
rows = cur_p2.fetchall()
fim = time.time()
tempo_consulta_100k = fim - inicio
print(f"Tempo de consulta (v = 5) com 100k tuplas: {tempo_consulta_100k:.6f} s")
print("Tuplas retornadas:", len(rows))

# Recriar tabela t com 1.000.000 de tuplas
print("\nRecriando tabela t com 1.000.000 tuplas...")

cur_p2.execute("DROP TABLE IF EXISTS t;")
cur_p2.execute("""
CREATE TABLE t (
    k serial PRIMARY KEY,
    v integer
);
""")
conn_p2.commit()

cur_p2.execute("""
INSERT INTO t(v)
SELECT trunc(random() * 10)
FROM generate_series(1,1000000);
""")
conn_p2.commit()
print("Tabela t populada com 1.000.000 de tuplas.")

inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v ON t(v);")
conn_p2.commit()
fim = time.time()
tempo_criacao_1m = fim - inicio
print(f"Tempo para criar índice idx_v (1M tuplas): {tempo_criacao_1m:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
rows = cur_p2.fetchall()
fim = time.time()
tempo_consulta_1m = fim - inicio
print(f"Tempo de consulta (v = 5) com 1M tuplas: {tempo_consulta_1m:.6f} s")
print("Tuplas retornadas:", len(rows))

Tempo para criar índice idx_v (100k tuplas): 0.092715 s
Tempo de consulta (v = 5) com 100k tuplas: 0.005964 s
Tuplas retornadas: 9969

Recriando tabela t com 1.000.000 tuplas...
Tabela t populada com 1.000.000 de tuplas.
Tempo para criar índice idx_v (1M tuplas): 0.431394 s
Tempo de consulta (v = 5) com 1M tuplas: 0.077012 s
Tuplas retornadas: 100391


- O tempo para criar o índice cresce aproximadamente de forma linear com o número de tuplas.

- O índice acelera consultas para v = 5, mantendo tempo < 0,1s mesmo com 1 milhão de tuplas.

### Tarefa 9 – Fillfactor

In [23]:
cur_p2.execute("DROP INDEX IF EXISTS idx_v;")
conn_p2.commit()

fillfactors = [60, 80, 90, 100]
resultados_ff = []

for ff in fillfactors:
    idx_name = f"idx_v_ff{ff}"
    cur_p2.execute(f"DROP INDEX IF EXISTS {idx_name};")
    conn_p2.commit()

    inicio = time.time()
    cur_p2.execute(f"""
        CREATE INDEX {idx_name} ON t(v)
        WITH (fillfactor = {ff});
    """)
    conn_p2.commit()
    fim = time.time()
    tempo_criacao = fim - inicio

    inicio = time.time()
    cur_p2.execute("SELECT * FROM t WHERE v = 5;")
    cur_p2.fetchall()
    fim = time.time()
    tempo_consulta = fim - inicio

    resultados_ff.append([ff, tempo_criacao, tempo_consulta])

df_fillfactor = pd.DataFrame(resultados_ff, columns=["fillfactor", "tempo_criacao", "tempo_consulta"])
print("\nResultados por fillfactor (índice ASC):")
print(df_fillfactor)


Resultados por fillfactor (índice ASC):
   fillfactor  tempo_criacao  tempo_consulta
0          60       0.468262        0.123255
1          80       0.501946        0.090838
2          90       0.482103        0.094269
3         100       0.435498        0.056962


Os resultados mostram que o fillfactor tem impacto mínimo no desempenho de consultas com igualdade (v = 5) em uma tabela estática. Isso ocorre porque o fillfactor afeta principalmente operações de escrita (inserções e atualizações) e não operações de leitura. Portanto, é esperado que todos os valores testados apresentem tempos muito semelhantes.

### Tarefa 10 – Índices DESC

In [24]:

cur_p2.execute("DROP INDEX IF EXISTS idx_v_desc;")
conn_p2.commit()

inicio = time.time()
cur_p2.execute("CREATE INDEX idx_v_desc ON t(v DESC NULLS FIRST);")
conn_p2.commit()
fim = time.time()
tempo_criacao_desc = fim - inicio
print(f"Tempo criação índice descendente idx_v_desc: {tempo_criacao_desc:.6f} s")

inicio = time.time()
cur_p2.execute("SELECT * FROM t WHERE v = 5;")
cur_p2.fetchall()
fim = time.time()
tempo_consulta_desc = fim - inicio
print(f"Tempo consulta (v = 5) com índice descendente: {tempo_consulta_desc:.6f} s")

fillfactors = [60, 80, 90, 100]
resultados_desc_ff = []

for ff in fillfactors:
    idx_name = f"idx_v_desc_ff{ff}"
    cur_p2.execute(f"DROP INDEX IF EXISTS {idx_name};")
    conn_p2.commit()

    inicio = time.time()
    cur_p2.execute(f"""
        CREATE INDEX {idx_name} ON t(v DESC NULLS FIRST)
        WITH (fillfactor = {ff});
    """)
    conn_p2.commit()
    fim = time.time()
    tempo_criacao = fim - inicio

    inicio = time.time()
    cur_p2.execute("SELECT * FROM t WHERE v = 5;")
    cur_p2.fetchall()
    fim = time.time()
    tempo_consulta = fim - inicio

    resultados_desc_ff.append([ff, tempo_criacao, tempo_consulta])

df_desc_fillfactor = pd.DataFrame(resultados_desc_ff, columns=["fillfactor", "tempo_criacao", "tempo_consulta"])
print("\nResultados por fillfactor (índice DESC):")
print(df_desc_fillfactor)

# Fechar conexão da Parte II
cur_p2.close()
conn_p2.close()
print("\nConexão da PARTE II encerrada.")

Tempo criação índice descendente idx_v_desc: 0.460955 s
Tempo consulta (v = 5) com índice descendente: 0.082836 s

Resultados por fillfactor (índice DESC):
   fillfactor  tempo_criacao  tempo_consulta
0          60       0.478406        0.106885
1          80       0.500126        0.056871
2          90       0.417167        0.057789
3         100       0.432121        0.054496

Conexão da PARTE II encerrada.


- Índices descendentes apresentam desempenho muito semelhante aos índices em ordem padrão (ASC) para consultas por igualdade (v = 5).

- A escolha entre ASC e DESC é mais importante quando o plano de execução precisa retornar dados ordenados por v, especialmente com ORDER BY v DESC LIMIT n.

- As variações de fillfactor (60, 80, 90, 100) nos índices descendentes não alteraram de forma significativa o tempo de consulta, o que confirma que:

    - fillfactor impacta principalmente operações de escrita,

    - enquanto consultas de leitura simples são pouco sensíveis a essa configuração em tabelas estáticas.

## Parte IV

O objetivo desta parte do trabalho é experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas. 


In [25]:
DB_CONFIG = {
    "dbname": "tpch",
    "user": "postgres",
    "password": "test123",
    "host": "localhost",
    "port": 5432
}

def reset_db():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS assentos;")
    cur.execute("""
        CREATE TABLE assentos(
            num_voo INT PRIMARY KEY,
            disp BOOLEAN
        );
    """)

    # 200 assentos inicialmente livres
    cur.executemany("INSERT INTO assentos VALUES (%s, true);", [(i,) for i in range(1, 201)])

    conn.commit()
    cur.close()
    conn.close()

reset_db()
print("Tabela assentos pronta.")


Tabela assentos pronta.


### Tarefa 17 – Código das Versões A e B

Versão A - Uma única transação

In [26]:
def reserva_versaoA(isolation_level):

    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_session(isolation_level=isolation_level)
    cur = conn.cursor()

    tentativas = 0

    while True:
        tentativas += 1
        try:
            # Passo 1 — escolher 1 assento livre e BLOQUEAR apenas ele
            cur.execute("""
                SELECT num_voo 
                FROM assentos 
                WHERE disp = true 
                ORDER BY random()
                LIMIT 1
                FOR UPDATE;
            """)
            row = cur.fetchone()

            # Nenhum assento livre → fim
            if not row:
                conn.rollback()
                break

            escolhido = row[0]

            # Passo 2 — espera de 1s (cliente escolhendo)
            time.sleep(1)

            # Passo 3 — marcar como ocupado
            cur.execute("""
                UPDATE assentos 
                SET disp = false 
                WHERE num_voo = %s;
            """, (escolhido,))

            conn.commit()
            break  # reserva feita com sucesso

        except psycopg2.errors.SerializationFailure:
            # conflitos serializáveis → tentar de novo
            conn.rollback()
            continue

        except psycopg2.Error as e:
            # erros inesperados devem ser reportados
            conn.rollback()
            print("Erro inesperado na versão A:", e)
            break

    cur.close()
    conn.close()
    return tentativas


Versão b - Duas transações

In [27]:
def reserva_versaoB(isolation_level):

    tentativas = 0

    while True:
        tentativas += 1

        try:
            # ================================
            # TRANSAÇÃO 1: obter 1 assento livre
            # ================================
            conn1 = psycopg2.connect(**DB_CONFIG)
            conn1.set_session(isolation_level=isolation_level)
            cur1 = conn1.cursor()

            cur1.execute("""
                SELECT num_voo 
                FROM assentos 
                WHERE disp = true 
                ORDER BY random()
                LIMIT 1;
            """)

            row = cur1.fetchone()

            if not row:
                conn1.commit()
                conn1.close()
                break  # nenhum assento disponível

            escolhido = row[0]

            conn1.commit()
            conn1.close()

            # ================================
            # PASSO 2 — escolha do cliente
            # ================================
            time.sleep(1)

            # ================================
            # TRANSAÇÃO 2: tentar reservar
            # ================================
            conn2 = psycopg2.connect(**DB_CONFIG)
            conn2.set_session(isolation_level=isolation_level)
            cur2 = conn2.cursor()

            cur2.execute("""
                UPDATE assentos
                SET disp = false
                WHERE num_voo = %s AND disp = true;
            """, (escolhido,))

            if cur2.rowcount == 1:
                conn2.commit()
                conn2.close()
                break  # reserva concluída
            else:
                conn2.rollback()
                conn2.close()
                continue  # alguém já pegou → tentar novamente

        except psycopg2.errors.SerializationFailure:
            # conflito serializável → repetir tentativa
            continue

        except Exception as e:
            # erro inesperado → reportar e quebrar
            print("Erro inesperado na versão B:", e)
            break

    return tentativas


### Tarefa 18 – Executar experimentos e gerar gráficos

In [ ]:
import time
import threading
import matplotlib.pyplot as plt
import numpy as np

# k usados no experimento
k_values = [1, 2, 4, 6, 8, 10]

# estrutura para armazenar os tempos de cada experimento
resultados = {
    "A_READ_COMMITTED": [],
    "A_SERIALIZABLE": [],
    "B_READ_COMMITTED": [],
    "B_SERIALIZABLE": []
}

estatisticas_tentativas = []

def executar_experimento(k, versao_func, isolation_level, num_clientes=200):
    """
    Executa um experimento com:
    - k threads (agentes de viagem)
    - num_clientes clientes (default = 200)
    - versao_func: reserva_versaoA ou reserva_versaoB
    - isolation_level: 'READ COMMITTED' ou 'SERIALIZABLE'

    Retorna:
        tempo_total (float),
        tentativas_total (lista de tentativas por cliente),
        estatisticas (dict com Min, Max, Média)
    """

    # Começa sempre com a tabela de 200 assentos livres
    reset_db()

    inicio = time.time()
    tentativas_total = []

    contador = {"cliente": 0}
    lock = threading.Lock()

    def worker():
        while True:
            # pegar "um cliente" para esta thread
            with lock:
                if contador["cliente"] >= num_clientes:
                    return
                contador["cliente"] += 1

            # atende um cliente (uma reserva completa)
            tentativas = versao_func(isolation_level)
            tentativas_total.append(tentativas)

    # cria k threads/agentes
    threads = [threading.Thread(target=worker) for _ in range(k)]

    # inicia e aguarda todas
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    tempo_total = time.time() - inicio

    # ---- estatísticas das tentativas ----
    arr = np.array(tentativas_total)
    estatisticas = {
        "Min": arr.min(),
        "Max": arr.max(),
        "Média": arr.mean()
    }

    return tempo_total, tentativas_total, estatisticas



Executar Experimentos

In [ ]:
for k in k_values:
    # Versão A - Read Committed
    t, tent, stats = executar_experimento(k, reserva_versaoA, "READ COMMITTED")
    resultados["A_READ_COMMITTED"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "A",
        "Isolamento": "READ COMMITTED",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão A - Serializable
    t, tent, stats = executar_experimento(k, reserva_versaoA, "SERIALIZABLE")
    resultados["A_SERIALIZABLE"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "A",
        "Isolamento": "SERIALIZABLE",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão B - Read Committed
    t, tent, stats = executar_experimento(k, reserva_versaoB, "READ COMMITTED")
    resultados["B_READ_COMMITTED"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "B",
        "Isolamento": "READ COMMITTED",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


In [ ]:
for k in k_values:
    # Versão B - Serializable
    t, tent, stats = executar_experimento(k, reserva_versaoB, "SERIALIZABLE")
    resultados["B_SERIALIZABLE"].append(t)

    estatisticas_tentativas.append({
        "k": k,
        "Versão": "B",
        "Isolamento": "SERIALIZABLE",
        "Min": stats["Min"],
        "Max": stats["Max"],
        "Média": stats["Média"]
    })


Gerar Gráficos

In [ ]:
def gerar_grafico(nome, valores, titulo):
    plt.figure(figsize=(8,5))
    plt.plot(k_values, valores, marker='o')
    plt.xlabel("Número de agentes (k)")
    plt.ylabel("Tempo total (s)")
    plt.title(titulo)
    plt.grid(True)

    # Range fixo para facilitar comparação entre gráficos 
    plt.ylim(0, 300)  

    plt.show()


gerar_grafico(
    "A_READ_COMMITTED",
    resultados["A_READ_COMMITTED"],
    "Versão A – Read Committed"
)

gerar_grafico(
    "A_SERIALIZABLE",
    resultados["A_SERIALIZABLE"],
    "Versão A – Serializable"
)

gerar_grafico(
    "B_READ_COMMITTED",
    resultados["B_READ_COMMITTED"],
    "Versão B – Read Committed"
)

gerar_grafico(
    "B_SERIALIZABLE",
    resultados["B_SERIALIZABLE"],
    "Versão B – Serializable"
)


### Tarefa 19 — Tabela de tentativas

In [ ]:
import pandas as pd

# Criar DataFrame final
df_tentativas = pd.DataFrame(estatisticas_tentativas)

# Ordenar 
df_tentativas = df_tentativas.sort_values(by=["Versão", "Isolamento", "k"])

df_tentativas


### Tarefa 20 - Análise dos Resultados




1. Versão A — Read Committed

Na versão A com Read Committed, todos os clientes concluíram a reserva sempre na primeira tentativa.

(Min = Max = Média = 1)

Isso ocorre porque:

- A leitura, a espera de 1 segundo e a escrita são executadas dentro de uma única transação.

- No nível Read Committed, a leitura não é bloqueada e o SGBD não precisa garantir serialização global.

- Conflitos reais só acontecem na escrita, e a chance de dois agentes reservarem o mesmo assento simultaneamente é baixa.

*Conclusão*: desempenho estável, sem retrabalho e tempos compatíveis com o aumento de k.


2. Versão A — Serializable

Este foi o cenário mais custoso da simulação. O número de tentativas aumentou significativamente conforme k cresceu, chegando a até 8 retries.

Motivos:

- O nível Serializable exige que a transação inteira seja equivalente a uma execução isolada.

- A versão A mantém a transação aberta durante o tempo de espera de 1 segundo, aumentando a janela crítica.

- Vários agentes leem o mesmo estado dos assentos e tentam reservar posições semelhantes.

- O PostgreSQL detecta conflitos de serialização (como phantom reads e write-write conflicts) e aborta transações, obrigando novas tentativas.

*Conclusão:* alta contenção, muitos abortos de transação e pior desempenho entre todas as combinações.

3. Versão B — Read Committed

A versão B apresentou apenas uma tentativa por cliente em todos os valores de k.

Isso acontece porque:

- A leitura e a escrita acontecem em transações separadas, ambas muito curtas.

- O tempo de espera de 1 segundo ocorre fora de qualquer transação, reduzindo drasticamente a chance de conflito.

- A escrita é rápida o suficiente para não gerar bloqueios significativos.

*Conclusão:* excelente desempenho e nenhuma tentativa extra, mesmo com alta concorrência.

4. Versão B — Serializable

Mesmo sob Serializable, a versão B manteve Min = Max = Média = 1.
Ou seja, a serialização não gerou overhead significativo.

Razões:

- Como a transação de escrita é extremamente curta, as chances de conflito são mínimas.

- A separação entre leitura e escrita impede phantoms e reduz o risco de abortos.

*Conclusão:* comportamento extremamente eficiente, mostrando que a versão B é robusta mesmo no nível de isolamento mais forte.